In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from skopt.plots import plot_objective, plot_convergence
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from preprocess import get_data


In [ ]:
trainData =get_data("train/","ascii_file_counts.csv")
testData = get_data("test","ascii_file_counts.csv")
XTrain, ytrain = trainData
XTest, ytest = testData